## Testing variable-gateset-dimension GST with model selection

###     Setup

In [1]:
import pygsti
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
#Load gateset and some string lists
gs_target = pygsti.io.load_gateset("tutorial_files/Example_Gateset.txt")
fiducialList = pygsti.io.load_gatestring_list("tutorial_files/Example_FiducialList.txt")
germList = pygsti.io.load_gatestring_list("tutorial_files/Example_GermsList.txt")
specs = pygsti.construction.build_spam_specs(fiducialList)
expList = [1,2,4]


In [5]:
#Create some testing gate string lists
lgstList = pygsti.construction.list_lgst_gatestrings(specs, gs_target.gates.keys())
lsgstLists = [ lgstList[:] ]
for exp in expList:
    gsList = pygsti.construction.create_gatestring_list(
                "f0+germ*exp+f1", f0=fiducialList, f1=fiducialList,
                germ=germList, exp=exp, order=['germ','f0','f1'])
    lsgstLists.append( lsgstLists[-1] +  gsList )
    
dsList = pygsti.remove_duplicates( lsgstLists[-1] )

In [9]:
#Test on fake data by depolarizing target set, increasing it's dimension,
# and adding leakage to the gates into the new dimension.

gs_dataGen4 = gs_target.depolarize(gate_noise=0.1)
gs_dataGen5 = gs_dataGen4.increase_dimension(5)
leakGate = pygsti.construction.build_gate( [2,1],[('Q0',),('L0',)] , "LX(pi/4.0,0,2)","gm") # X(pi,Q0)*LX(pi,0,2)

gs_dataGen5['Gx'] = pygsti.objects.compose( gs_dataGen5['Gx'], leakGate)
gs_dataGen5['Gy'] = pygsti.objects.compose( gs_dataGen5['Gy'], leakGate)
print gs_dataGen5.gates.keys()

#Some debugging...
#NOTE: with LX(pi,0,2) above, dim 5 test will choose a dimension 3 gateset, which may be sensible
#       looking at the gate matrices in this case... but maybe LX(pi,...) is faulty?
#print gs_dataGen4
#print gs_dataGen5

#Jmx = GST.JOps.jamiolkowski_iso(gs_dataGen4['Gx'])
#Jmx = GST.JOps.jamiolkowski_iso(gs_dataGen5['Gx'],dimOrStateSpaceDims=[2,1])
#print "J = \n",Jmx
#print "evals = ",eigvals(Jmx)

dsFake4 = pygsti.construction.generate_fake_data(gs_dataGen4, dsList, nSamples=1000000, sampleError="binomial", seed=1234)
dsFake5 = pygsti.construction.generate_fake_data(gs_dataGen5, dsList, nSamples=1000000, sampleError="binomial", seed=1234)

['Gi', 'Gx', 'Gy']


In [11]:
print "Number of gates = ",len(gs_target.gates.keys())
print "Number of fiducials =",len(fiducialList)
print "Maximum length for a gate string in ds =",max(map(len,dsList))
print "Number of LGST strings = ",len(lgstList)
print "Number of LSGST strings = ",map(len,lsgstLists)

 Number of gates =  3
Number of fiducials = 6
Maximum length for a gate string in ds = 30
Number of LGST strings =  92
Number of LSGST strings =  [92, 488, 884, 1280]


### Test using dimension-4 fake data

In [12]:
#Run LGST to get an initial estimate for the gates in gs_target based on the data in ds
# NOTE: with nSamples less than 1M (100K, 10K, 1K) this routine will choose a higher-than-4 dimensional gateset
ds = dsFake4
gs_lgst4 = pygsti.do_lgst(ds, specs, targetGateset=gs_target, svdTruncateTo=4, verbosity=3)
gs_lgst6 = pygsti.do_lgst(ds, specs, targetGateset=gs_target, svdTruncateTo=6, verbosity=3)

#Print chi^2 of 4-dim and 6-dim estimates
chiSq4 = pygsti.chi2(ds, gs_lgst4, lgstList, minProbClipForWeighting=1e-4)
chiSq6 = pygsti.chi2(ds, gs_lgst6, lgstList, minProbClipForWeighting=1e-4)
print "LGST dim=4 chiSq = ",chiSq4
print "LGST dim=6 chiSq = ",chiSq6

# Least squares GST with model selection
gs_lsgst = pygsti.do_iterative_mc2gst_with_model_selection(ds, gs_lgst4, 1, lsgstLists, verbosity=2,
                                                           minProbClipForWeighting=1e-3, probClipInterval=(-1e5,1e5))

LGST: Singular values of I_tilde (truncating to first 4 of 6) = 
[  3.00484791e+00   8.25822588e-01   6.70903966e-01   6.65330643e-01
   1.32123551e-03   7.62101431e-04]

--- LGST ---
LGST: Singular values of I_tilde (truncating to first 6 of 6) = 
[  3.00484791e+00   8.25822588e-01   6.70903966e-01   6.65330643e-01
   1.32123551e-03   7.62101431e-04]
LGST: Padding target B with sqrt of low singular values of I_tilde: 
[ 0.00132124  0.0007621 ]

--- LGST ---
LGST dim=4 chiSq =  584.010103572
LGST dim=6 chiSq =  218.874929215

--- Iterative LSGST: Beginning iter 1 of 4 : 92 gate strings ---
--- Least Squares GST with model selection (starting dim = 4) ---
--- Least Squares GST ---
  Sum of Chi^2 = 58.5712 (92 data params - 40 model params = expected mean of 52; p-value = 0.247042)
Dim 4: chi^2 = 58.5712, nGateStrings=92, nParams=56 (so expected mean = 36)
--- Least Squares GST ---
  Sum of Chi^2 = 5.5151e+06 (92 data params - 24 model params = expected mean of 68; p-value = 0)
Rejected 

In [13]:
print gs_lsgst

rho0 =    0.7071   0.0242  -0.0257   0.7454


E0 =    0.6852  -0.0149   0.0109  -0.6492


Gi = 
   1.0000        0        0        0
   0.0030   0.9001  -0.0001   0.0003
  -0.0028        0   0.8998   0.0002
  -0.0036  -0.0001   0.0002   0.9000


Gx = 
   1.0000        0        0        0
   0.0027   0.8993   0.0119  -0.0190
   0.0044   0.0243  -0.0108   0.9823
  -0.0578  -0.0184  -0.8249   0.0112


Gy = 
   1.0000        0        0        0
  -0.0041  -0.0197   0.0023  -0.9987
  -0.0025  -0.0077   0.8998   0.0041
  -0.0579   0.8113   0.0281   0.0197





### Test using dimension-5 fake data

In [14]:
#Run LGST to get an initial estimate for the gates in gs_target based on the data in ds
ds = dsFake5
gs_lgst4 = pygsti.do_lgst(ds, specs, targetGateset=gs_target, svdTruncateTo=4, verbosity=3)
gs_lgst6 = pygsti.do_lgst(ds, specs, targetGateset=gs_target, svdTruncateTo=6, verbosity=3)

#Print chi^2 of 4-dim and 6-dim estimates
chiSq4 = pygsti.chi2(ds, gs_lgst4, lgstList, minProbClipForWeighting=1e-2)
chiSq6 = pygsti.chi2(ds, gs_lgst6, lgstList, minProbClipForWeighting=1e-2)
print "LGST dim=4 chiSq = ",chiSq4
print "LGST dim=6 chiSq = ",chiSq6

# Least squares GST with model selection
gs_lsgst = pygsti.do_iterative_mc2gst_with_model_selection(ds, gs_lgst4, 1, lsgstLists, verbosity=2, minProbClipForWeighting=1e-3, probClipInterval=(-1e5,1e5), useFreqWeightedChiSq=False, regularizeFactor=1.0, check=False, check_jacobian=False)

LGST: Singular values of I_tilde (truncating to first 4 of 6) = 
[  2.36368304e+00   6.59319496e-01   4.68883491e-01   4.54819490e-01
   3.15418560e-03   8.91359166e-04]

--- LGST ---
LGST: Singular values of I_tilde (truncating to first 6 of 6) = 
[  2.36368304e+00   6.59319496e-01   4.68883491e-01   4.54819490e-01
   3.15418560e-03   8.91359166e-04]
LGST: Padding target B with sqrt of low singular values of I_tilde: 
[ 0.00315419  0.00089136]

--- LGST ---
LGST dim=4 chiSq =  1316593.37275
LGST dim=6 chiSq =  580943.33613

--- Iterative LSGST: Beginning iter 1 of 4 : 92 gate strings ---
--- Least Squares GST with model selection (starting dim = 4) ---
--- Least Squares GST ---
  Sum of Chi^2 = 143358 (92 data params - 40 model params = expected mean of 52; p-value = 0)
Dim 4: chi^2 = 143358, nGateStrings=92, nParams=56 (so expected mean = 36)
--- Least Squares GST ---
  Sum of Chi^2 = 3.0845e+06 (92 data params - 24 model params = expected mean of 68; p-value = 0)
Rejected dim 3: chi

In [15]:
print gs_lsgst

rho0 =    0.7070  -0.0472   0.0182   0.7723  -0.0021   0.0048


E0 =    0.5834  -0.0124  -0.0049  -0.5350  -0.0375   0.0363


Gi = 
   1.0000  -0.0001        0        0        0  -0.0021
  -0.0081   0.8960   0.0013   0.0005   0.0185  -0.0399
   0.0026   0.0029   0.8979   0.0003  -0.0209   0.0047
  -0.0240   0.0038   0.0039   0.8998   0.0348   0.1677
  -0.0018  -0.0014   0.0089  -0.0004   0.9974   0.2082
   0.0112  -0.0605  -0.0060   0.0014   0.0288  -0.3300


Gx = 
   1.0000  -0.0003        0        0   0.0002  -0.0058
   0.0297   0.8404  -0.0246   0.0375  -0.0295   0.0889
   0.1009   0.0760  -0.0320   0.9261  -0.4232  -0.1529
  -0.1182  -0.0446  -0.7112   0.0576   0.1401  -0.0861
   0.2417   0.0091   0.0581   0.0848   0.7509  -0.0676
   0.0051  -0.1034   0.0031   0.0020   0.0655  -1.1053


Gy = 
   1.0000  -0.0002        0        0        0  -0.0027
  -0.1633  -0.0765   0.0452  -0.9393   0.4060  -0.0821
  -0.0473  -0.0097   0.8331  -0.0358   0.0261  -0.0418
  -0.0748   0.7059  -0.0050